In [1]:
#ipcluster start -n=8

In [1]:
%pylab inline
import numexpr as ne
import seaborn as sns
from ipywidgets import interact
import time

Populating the interactive namespace from numpy and matplotlib


In [2]:
ne.set_num_threads(32)

8

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
%gui qt

In [5]:
import sys
sys.path.append('../../..')

In [6]:
from spiketag.mvc import Sorter

In [9]:
nCh=32
fs=25000
app = Sorter('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie.bin',nCh=nCh, fs=fs, ch_span=1, 
                                      corr_cutoff=0.95, 
                                      fetlen=3, fet_whiten=False,
                                      fall_off_size=18, n_jobs=2)

[INFO/MainProcess] load mua data


#############  load data  ###################
/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie.bin loaded, it contains: 
30150587 * 32 points (3859275136 bytes) 
32 channels with sampling rate of 25000.0000 
1206.023 secs (20.100 mins) of data
#############################################


[INFO/MainProcess] removing high corr noise from spikes pool
[INFO/MainProcess] extract spikes from pivital meta data
[INFO/MainProcess] extrat features with weighted-pca
[INFO/MainProcess] clustering with hdbscan
[INFO/PoolWorker-1] child process calling self.run()
[INFO/PoolWorker-2] child process calling self.run()
/Users/chiy/anaconda/lib/python2.7/site-packages/hdbscan/hdbscan_.py:281: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  n_jobs=core_dist_n_jobs, **kwargs)
/Users/chiy/anaconda/lib/python2.7/site-packages/hdbscan/hdbscan_.py:281: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  n_jobs=core_dist_n_jobs, **kwargs)
[INFO/PoolWorker-2] process shutting down
[INFO/PoolWorker-1] process shutting down
[INFO/PoolWorker-1] process exiting with exitcode 0
[INFO/PoolWorker-2] process exiting with exitcode 0


clustering finished, used 8.52346396446 seconds


[INFO/MainProcess] Model.spktag is generated, ch_span:1, nspk:326834


In [10]:
app.run()

please do origin_point first!
please do origin_point first!
please do origin_point first!
please do origin_point first!
please do origin_point first!
please do origin_point first!
please do origin_point first!
please do origin_point first!
please do origin_point first!
please do origin_point first!
please do origin_point first!
please do origin_point first!


  File "/Users/chiy/anaconda/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/chiy/anaconda/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/chiy/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/chiy/anaconda/lib/python2.7/site-packages/traitlets/config/application.py", line 653, in launch_instance
    app.start()
  File "/Users/chiy/anaconda/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/Users/chiy/anaconda/lib/python2.7/site-packages/zmq/eventloop/ioloop.py", line 162, in start
    super(ZMQIOLoop, self).start()
  File "/Users/chiy/anaconda/lib/python2.7/site-packages/tornado/ioloop.py", line 831, in start
    self._run_callback(callback)
  File "/Users/chiy/anaconda/lib/python2.7/site-packages/tornado/ioloop.py", line 604, in _run_c

please do origin_point first!
please do origin_point first!
please do origin_point first!
please do origin_point first!
please do origin_point first!
please do origin_point first!
please do origin_point first!
please do origin_point first!
please do origin_point first!


In [20]:
app.view.spk_view.highlight({1:np.array([0,1]),2:np.array([0,1])})A

In [ ]:
app.view.spk_view

In [10]:
app.view.spk_view._selected

{2: array([  0,   1,   5,   6,   7,   8,   9,  11,  12,  13,  15,  17,  20,
         21,  23,  25,  27,  28,  29,  30,  31,  32,  33,  34,  37,  38,
         39,  40,  42,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,
         54,  55,  56,  57,  58,  59,  61,  62,  63,  64,  65,  66,  69,
         73,  74,  75,  76,  77,  78,  79,  80,  81,  83,  85,  98,  99,
        100, 105, 106, 108, 115, 116, 120, 122, 124, 125, 126, 127, 128,
        131, 136, 139, 143, 144, 145, 146, 147, 148, 150, 151, 153, 154,
        156, 157, 158, 159, 160, 162, 163, 165, 169, 170, 171, 172, 173,
        179, 180, 182, 183, 186, 187, 189, 190, 194, 196, 199, 202, 205,
        206, 209, 210, 211, 213, 218, 219, 223, 225, 229, 233, 237, 239,
        240, 243, 244, 245, 246, 248, 250, 251, 252, 256, 257, 258, 259,
        260, 261, 263, 265, 267, 269, 270, 272, 273, 274, 275, 277, 279,
        285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297,
        299, 309, 311, 315, 316, 317, 318, 319, 

In [64]:
app.view.spk_view.clu.membership

5

In [11]:
app.view.spk_view.clu.index

{0: array([   53,    67,    77,    94,    96,   124,   134,   139,   245,
          276,   277,   521,   537,   646,   655,  1058,  1072,  1101,
         1108,  1152,  1638,  1734,  1758,  1846,  1850,  1859,  1862,
         1891,  1997,  2258,  2500,  2552,  2629,  2701,  2744,  2745,
         2774,  2793,  2830,  2872,  2885,  2903,  2933,  2936,  2939,
         2943,  2945,  2950,  2997,  3008,  3031,  3061,  3077,  3100,
         3130,  3147,  3178,  3218,  3232,  3275,  3290,  3295,  3303,
         3334,  3347,  3349,  3365,  3366,  3421,  3465,  3497,  3499,
         3516,  3521,  3525,  3528,  3585,  3588,  3598,  3615,  3634,
         3640,  3650,  3659,  3665,  3679,  3688,  3752,  3764,  3765,
         3768,  3769,  3794,  3832,  3868,  3879,  3884,  3907,  3952,
         3980,  3997,  4044,  4046,  4050,  4086,  4164,  4245,  4248,
         4249,  4261,  4270,  4279,  4324,  4342,  4346,  4373,  4391,
         4475,  4489,  4536,  4572,  4598,  4702,  4729,  4740,  4758,
   

In [35]:
app.view.spk_view._cache_mask_

741

In [48]:
app.model.clu[26].index[1]

array([    1,    16,    21, ..., 12516, 12660, 12707])

In [89]:
app.view.spk_view.clu.local2global(app.view.spk_view._selected)

array([ 162,  226,  372,  436,  448,  453,  549,  552,  723,  729,  869,
        874,  945,  948,  971,  976, 1071, 1119, 1130, 1164, 1174, 1190,
       1261, 1272, 1273, 1290, 1370, 1379, 1474, 1499, 1589, 1630, 1658,
       1671, 1690, 1736, 1769, 1931, 1976, 2043, 2199, 2292, 2347, 2384,
       2405])

In [90]:
#app.view.spk_view.clu.local2global(app.view.spk_view._selected).all() in app.model.clu[26].index[1]
for i in [ 162,  226,  372,  436,  448,  453,  549,  552,  723,  729,  869,
        874,  945,  948,  971,  976, 1071, 1119, 1130, 1164, 1174, 1190,
       1261, 1272, 1273, 1290, 1370, 1379, 1474, 1499, 1589, 1630, 1658,
       1671, 1690, 1736, 1769, 1931, 1976, 2043, 2199, 2292, 2347, 2384,
       2405]:
    if i not in list(app.model.clu[26].index[3]):
        print('byte')


In [62]:
app.model.clu[26].index

{0: array([   53,    77,   124,   134,   139,   146,   208,   245,   276,
          281,   407,   521,   537,   646,  1098,  1101,  1108,  1638,
         1846,  1850,  1859,  1862,  1891,  2500,  2552,  2744,  2745,
         2793,  2830,  2872,  2885,  2903,  2933,  2936,  2939,  2943,
         2945,  2950,  2997,  3008,  3031,  3061,  3077,  3100,  3130,
         3147,  3178,  3218,  3275,  3290,  3295,  3303,  3334,  3347,
         3349,  3365,  3366,  3421,  3465,  3497,  3499,  3516,  3521,
         3525,  3585,  3588,  3615,  3640,  3650,  3659,  3665,  3679,
         3688,  3752,  3764,  3765,  3768,  3769,  3832,  3879,  3884,
         3907,  3952,  3980,  3997,  4044,  4046,  4086,  4164,  4245,
         4248,  4249,  4261,  4270,  4279,  4324,  4342,  4346,  4373,
         4391,  4489,  4536,  4572,  4598,  4729,  4740,  4844,  4855,
         4859,  4861,  4862,  4902,  4938,  4992,  5065,  5075,  5085,
         5086,  5087,  5122,  5138,  5182,  5259,  5296,  5331,  5351,
   

In [35]:
app.view.spk_view._spkNolist

{0,
 1,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 14,
 15,
 16,
 17,
 19,
 20,
 21,
 22,
 24,
 26,
 27,
 28,
 29,
 31,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 58,
 59,
 60,
 62,
 63,
 64,
 65,
 66,
 67,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 78,
 79,
 80,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 105,
 106,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 146,
 149}

In [37]:
app.view.spk_view._selected_cluster

1

In [10]:
app.model.clu[26].index

{0: array([   53,    67,    77,    94,    96,   124,   134,   139,   146,
          208,   245,   276,   277,   281,   407,   521,   537,   646,
          655,  1058,  1072,  1098,  1101,  1108,  1152,  1638,  1734,
         1758,  1846,  1850,  1859,  1862,  1891,  1997,  2258,  2500,
         2552,  2629,  2701,  2744,  2745,  2774,  2793,  2830,  2872,
         2885,  2903,  2933,  2936,  2939,  2943,  2945,  2950,  2997,
         3008,  3031,  3061,  3077,  3100,  3130,  3147,  3178,  3218,
         3232,  3275,  3290,  3295,  3303,  3334,  3347,  3349,  3365,
         3366,  3421,  3465,  3497,  3499,  3516,  3521,  3525,  3528,
         3585,  3588,  3598,  3615,  3634,  3640,  3650,  3659,  3665,
         3679,  3688,  3752,  3764,  3765,  3768,  3769,  3794,  3832,
         3868,  3879,  3884,  3907,  3952,  3980,  3997,  4044,  4046,
         4050,  4086,  4164,  4245,  4248,  4249,  4261,  4270,  4279,
         4324,  4342,  4346,  4373,  4391,  4475,  4489,  4536,  4572,
   